In [ ]:
import pandas as pd
import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
import os

In [ ]:
all_players = players.get_players()

In [ ]:
def APIToDataframe(players_list, start=0, end=None, sleep_time=1.0, start_counter=1):
    """
    Fetch NBA player stats for a slice of players, single attempt per player.
    """
    stats_list = []
    failed_players = []
    counter = start_counter

    players_slice = players_list[start:end]

    for player in players_slice:
        try:
            career = playercareerstats.PlayerCareerStats(player_id=player['id'], timeout=60).get_data_frames()
            time.sleep(sleep_time)

            if career and len(career) > 0 and not career[0].empty:
                regular_season_df = career[0]
                regular_season_df['player_id'] = player['id']
                stats_list.append(regular_season_df)
                print(f"{counter}. Added player_id: {player['id']} - {player['first_name']} {player['last_name']}")
            else:
                print(f"{counter}. No regular season data for player {player['id']} - {player['first_name']} {player['last_name']}")
                failed_players.append(player)

        except Exception as e:
            print(f"{counter}. Error fetching player {player['id']}: {e}")
            failed_players.append(player)

        counter += 1

    full_df = pd.concat(stats_list, ignore_index=True) if stats_list else pd.DataFrame()
    return full_df, failed_players, counter

In [ ]:
def RetryFailedPlayers(failed_players, sleep_time=1.0, start_counter=1):
    stats_list = []
    still_failed = []
    counter = start_counter

    for player in failed_players:
        try:
            career = playercareerstats.PlayerCareerStats(player_id=player['id'], timeout=60).get_data_frames()
            time.sleep(sleep_time)

            if career and len(career) > 0 and not career[0].empty:
                regular_season_df = career[0]
                regular_season_df['player_id'] = player['id']
                stats_list.append(regular_season_df)
                print(f"{counter}. Retry succeeded: {player['id']} - {player['first_name']} {player['last_name']}")
            else:
                print(f"{counter}. No data for player {player['id']} - {player['first_name']} {player['last_name']}")
                still_failed.append(player)

        except Exception as e:
            print(f"{counter}. Retry failed for player {player['id']}: {e}")
            still_failed.append(player)

        counter += 1

    retry_df = pd.concat(stats_list, ignore_index=True) if stats_list else pd.DataFrame()
    print(f"Retry complete: {len(stats_list)} succeeded, {len(still_failed)} still failed.")
    return retry_df, still_failed, counter

In [ ]:
def save_batch(batch_df, failed_players, start_index, end_index, master_csv="nba_stats_master.csv"):
    # Append batch to master CSV
    if os.path.exists(master_csv):
        batch_df.to_csv(master_csv, mode='a', header=False, index=False)
    else:
        batch_df.to_csv(master_csv, index=False)
    print(f"Batch {start_index}-{end_index} appended to master CSV ({len(batch_df)} rows)")

    # Save failed players as CSV
    if failed_players:
        failed_csv = f"failed_{start_index}_{end_index}.csv"
        failed_df = pd.DataFrame(failed_players)
        failed_df.to_csv(failed_csv, index=False)
        print(f"{len(failed_players)} failed players saved to {failed_csv}")
    else:
        print("No failed players in this batch.")

In [ ]:
start_index = 850
end_index = 1000

In [ ]:
batch_df, failed, counter = APIToDataframe(all_players, start=start_index, end=end_index)

In [ ]:
retry_df, still_failed, counter = RetryFailedPlayers(failed, start_counter=counter)

In [ ]:
combined_batch = pd.concat([batch_df, retry_df], ignore_index=True)

In [ ]:
save_batch(combined_batch, still_failed, start_index, end_index)